In [1]:
# 9장 

In [ ]:
# 인공지능 베이스 - 디지털 선호 처리

# 디지털 필터의 2가지 (어떤 개념을 적용하더라도 이 2가지 개념을 벗어날 수 없음)
# CNN - FIR 필터 (이전의 값에 영향 받지 않음) ->> 주가 분석 뿐만 아니라 음성 분석도 가능
# RNN, LSTM - IIR 필터 (이전 값이 현재 결과에 영향을 줌) ->> 주가 분석 뿐만 아니라 음성 분석도 가능
# 보통 이 2개 섞어서 씀!

# 피드백(IIR만 피드백 됨)을 준다는 개념은 특정값을 찾아 수렴시키는 것이라 보면 된다.
# 커드쿼터, 텍(헥)사쿼터 만든다고 하면 제일 중요한 것이 균형 맞추는 것!
# 특정값을 찾아 수렴시키는 것 ( = 헬기의 균형을 찾는 것과 같은 의미)
# 어떤 특정 위치에서 고정이 되어 있어야 함.
# 하지만, 센서데이터는 고정되어 있지 않으니까!!

# 주가에서는 이전에 어떤 이벤트, 영향이 있었나의 영향을 받음!! (현재의 결과에 영향이 있음!)
# 주식쪽으로 마이크로 펀딩 하고 싶다면 => FPGA 필요!!! (실시간으로 하드웨어 가속하기 위한 장치)

# 피드백을 주면서 노이즈가 끼게 됨.
# 노이즈가 끼게 되면서 안정적으로 센서 데이터를 받을 수 있는 것이 IIR임!
# 현재는 이름이 RNN, LTSM으로 많이 쓰고 있지만, 결국은 IIR 필터와 같음.

In [ ]:
# y(u, x) => ∂y/∂u 혹은 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!ls Applied-Deep-Learning-with-Keras/Lesson09

AAPL_test.csv	  Activity18.ipynb  Exercise28.ipynb
AAPL_train.csv	  Activity19.ipynb  MSFT_test.csv
Activity17.ipynb  Exercise27.ipynb  MSFT_train.csv


In [3]:
trainSet = pd.read_csv("Applied-Deep-Learning-with-Keras/Lesson09/AAPL_train.csv")
trainSet.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,02-01-2014,79.382858,79.575714,78.860001,79.018570,66.964325,58671200
1,03-01-2014,78.980003,79.099998,77.204285,77.282860,65.493416,98116900
2,06-01-2014,76.778572,78.114288,76.228569,77.704285,65.850533,103152700
3,07-01-2014,77.760002,77.994286,76.845711,77.148575,65.379593,79302300
4,08-01-2014,76.972855,77.937141,76.955711,77.637146,65.793633,64632400


In [4]:
trainSet = trainSet.iloc[:, 1:2].values
trainSet

array([[ 79.382858],
       [ 78.980003],
       [ 76.778572],
       ...,
       [155.839996],
       [157.5     ],
       [158.529999]])

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
sc = MinMaxScaler(feature_range = (0, 1))
trainSetScaled = sc.fit_transform(trainSet)
trainSetScaled

array([[0.05400437],
       [0.05148716],
       [0.03773165],
       ...,
       [0.53174205],
       [0.54211448],
       [0.54855037]])

In [9]:
X_train = []
y_train = []

for i in range(60, 1258):
    X_train.append(trainSetScaled[i - 60:i, 0])
    y_train.append(trainSetScaled[i, 0])
    
X_train = np.array(X_train)
y_train = np.array(y_train)

In [11]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train

array([[[0.05400437],
        [0.05148716],
        [0.03773165],
        ...,
        [0.04582785],
        [0.04002572],
        [0.03850823]],

       [[0.05148716],
        [0.03773165],
        [0.04386406],
        ...,
        [0.04002572],
        [0.03850823],
        [0.03932056]],

       [[0.03773165],
        [0.04386406],
        [0.03894562],
        ...,
        [0.03850823],
        [0.03932056],
        [0.03800839]],

       ...,

       [[0.98231691],
        [0.97794302],
        [0.99543867],
        ...,
        [0.53811549],
        [0.48369155],
        [0.48462887]],

       [[0.97794302],
        [0.99543867],
        [1.        ],
        ...,
        [0.48369155],
        [0.48462887],
        [0.53174205]],

       [[0.99543867],
        [1.        ],
        [0.98237946],
        ...,
        [0.48462887],
        [0.53174205],
        [0.54211448]]])

In [12]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [14]:
model = Sequential()

model.add(LSTM(50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50))
model.add(Dense(1))


In [15]:
model.compile(
    optimizer = 'adam',
    loss = 'mean_squared_error'
)

model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
38/38 [==============================] - 2s 47ms/step - loss: 0.0256
Epoch 2/100
38/38 [==============================] - 2s 44ms/step - loss: 0.0026
Epoch 3/100
38/38 [==============================] - 2s 43ms/step - loss: 0.0022
Epoch 4/100
38/38 [==============================] - 2s 44ms/step - loss: 0.0024
Epoch 5/100
38/38 [==============================] - 2s 43ms/step - loss: 0.0017
Epoch 6/100
38/38 [==============================] - 2s 43ms/step - loss: 0.0023
Epoch 7/100
38/38 [==============================] - 2s 42ms/step - loss: 0.0016
Epoch 8/100
38/38 [==============================] - 2s 44ms/step - loss: 0.0014
Epoch 9/100
38/38 [==============================] - 2s 43ms/step - loss: 0.0014
Epoch 10/100
38/38 [==============================] - 2s 43ms/step - loss: 0.0015
Epoch 11/100
38/38 [==============================] - 2s 44ms/step - loss: 0.0013
Epoch 12/100
38/38 [==============================] - 2s 43ms/step - loss: 0.0014
Epoch 13/100
38/38 [=====

38/38 [==============================] - 2s 42ms/step - loss: 2.3907e-04
Epoch 98/100
38/38 [==============================] - 2s 43ms/step - loss: 2.5773e-04
Epoch 99/100
38/38 [==============================] - 2s 42ms/step - loss: 3.0673e-04
Epoch 100/100
38/38 [==============================] - 2s 42ms/step - loss: 2.2823e-04
